In [0]:
rawPath = f"dbfs:/FileStore/tables/*.json"
bronzePath = f"/movie/bronzes10"
silverPath = f"/movie/silver10"
silverGenrePath = f"/movie/silver_genre10"
silverMovieGenrePath = f"/movie/silver_movie_genre10"
silverOriginalLanguagePath = f"/movie/silver_originalLanguage10"

In [0]:
def read_batch_raw(rawPath):
    return spark.read.option("multiLine", "true").json("dbfs:/FileStore/tables/*.json")


In [0]:
def read_batch_bronze(spark):
    return spark.read.table("movies_classic_bronze").filter("status= 'new'")

In [0]:
from pyspark.sql.functions import col, explode
from pyspark.sql.functions import current_timestamp, lit, to_json

def transform_raw(rawDF):
    rawDF = rawDF.withColumn('movie', explode('movie'))
    return rawDF.select(
        to_json(col("movie")).alias("movie"),
        lit("antra_movieshops").alias("datasource"),
        current_timestamp().alias("ingesttime"),
        lit("new").alias("status"),
        current_timestamp().cast("date").alias("ingestdate"),
     )
    

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, LongType, DateType, MapType
from pyspark.sql.functions import from_json, col

def transform_bronze(bronzeDF): 
    json_schema = StructType(fields=[
        StructField('BackdropUrl', StringType(), True),
        StructField('Budget', StringType(), True),
        StructField('CreatedDate', DateType(), True),
        StructField('Id', IntegerType(), True),
        StructField('ImdbUrl', StringType(), True),
        StructField('OriginalLanguage', StringType(), True),
        StructField('Overview', StringType(), True),
        StructField('PosterUrl', StringType(), True),
        StructField('Price', DoubleType(), True),
        StructField('ReleaseDate', StringType(), True),
        StructField('Revenue', DoubleType(), True),
        StructField('RunTime', DoubleType(), True),
        StructField('Tagline', StringType(), True),
        StructField('Title', StringType(), True),
        StructField('TmdbUrl', StringType(), True),
        StructField(
            'genres', ArrayType(
                StructType([
                    StructField('id', IntegerType(), True),
                    StructField('name', StringType(), True)
                ])
            )
        )
    ])

    bronzeAugmentedDF = bronzeDF.withColumn(
        "nested_json", from_json(col("movie"), json_schema)
    )
    
    silver_movies = bronzeAugmentedDF.select("movie", "nested_json.*")
    return silver_movies.select(
         'movie',
         'BackdropUrl',
         'Budget',
         'CreatedDate',
         'Id',
         'ImdbUrl',
         'OriginalLanguage',
         'Overview',
         'PosterUrl',
         'Price',
         col('ReleaseDate').alias('p_ReleasedDate'),
         'Revenue',
         'RunTime',
         'Tagline',
         'Title',
         'TmdbUrl',
         'genres',
    ).dropDuplicates()
    

In [0]:
def generate_clean_and_quarantine_dataframes(transformedBronzeDF: DataFrame):
    return (transformedBronzeDF.drop("genres").filter("RunTime >= 0"),
            transformedBronzeDF.drop("genres").filter("RunTime < 0")
    )

In [0]:
def generate_genre_silver(transformedBronzeDF: DataFrame):
    genres = transformedBronzeDF.select(
        "Id",
        "genres"
    )
    silver_genre_exploded = (genres.withColumn(
    "genre_json", explode("genres"))
                         .drop("genres")
                         .dropDuplicates()
                        )
    return silver_genre_exploded.select(
        col('genre_json.id').alias('genre_id'),
        col('genre_json.name').alias('genre_name')
    ).dropDuplicates(["genre_id"])

In [0]:
def generate_movie_genre_silver(transformedBronzeDF: DataFrame):
    genres = transformedBronzeDF.select(
        "Id",
        "genres"
    )
    
    silver_genre_exploded = (genres.withColumn(
        "genre_json", explode("genres"))
                             .drop("genres")
                             .dropDuplicates()
                            )
    return silver_genre_exploded.select(
        'Id',
        col('genre_json.id').alias('genre_id')
    ).dropDuplicates()

In [0]:
def generate_originalLanguage_silver(transformedBronzeDF: DataFrame):
    from pyspark.sql.functions import to_json
    languages = transformedBronzeDF.select(
        'OriginalLanguage'
    ).dropDuplicates()
    return languages.select(
        col('OriginalLanguage').alias('LanguageCode'),
        lit('English').alias('LanguageName')
    )

In [0]:
from delta.tables import DeltaTable
from pyspark.sql.session import SparkSession
def update_bronze_table_status(
    spark: SparkSession, bronzeTablePath: str, dataframe: DataFrame, status: str
) -> bool:

    bronzeTable = DeltaTable.forPath(spark, bronzePath)
    dataframeAugmented = dataframe.withColumn("status", lit(status))

    update_match = "bronze.movie = dataframe.movie"
    update = {"status": "dataframe.status"}

    (
        bronzeTable.alias("bronze")
        .merge(dataframeAugmented.alias("dataframe"), update_match)
        .whenMatchedUpdate(set=update)
        .execute()
    )

    return True

In [0]:
from pyspark.sql import DataFrame

def batch_writer_bronze(dataframe: DataFrame, partition_column: str) -> DataFrame:
    return(
        dataframe.select(
            "movie", 
            "datasource",
            "ingesttime",
            "status", 
            col("ingestdate").alias(partition_column),
        )
      .write.format("delta")
      .mode("append")
      .partitionBy(partition_column)
    )
    

In [0]:
def batch_writer_silver(dataframe: DataFrame, partition_column: str) -> DataFrame:
    return (dataframe.select("*").drop("movie")
      .write.format("delta")
      .mode("append")
      .partitionBy("p_ReleasedDate")
    )

In [0]:
def batch_writer_silver_genre_movie_language(dataframe: DataFrame) -> DataFrame:
    return (dataframe.select("*")
      .write.format("delta")
      .mode("append")
    )

# Raw to Bronze

In [0]:
rawDF = read_batch_raw(rawPath)
transformedRawDF = transform_raw(rawDF)
rawToBronzeWriter = batch_writer_bronze(
  dataframe=transformedRawDF, partition_column="p_ingestdate"
)

rawToBronzeWriter.save(bronzePath)

# Bronze to Silver

In [0]:
bronzeDF = read_batch_bronze(spark)
transformedBronzeDF = transform_bronze(bronzeDF)
(silverCleanDF, silverQuarantineDF) = generate_clean_and_quarantine_dataframes(
    transformedBronzeDF
)
silver_genre = generate_genre_silver(transformedBronzeDF)
silver_movie_genre = generate_movie_genre_silver(transformedBronzeDF)
silver_originalLanguage = generate_originalLanguage_silver(transformedBronzeDF)

bronzeToSilverWriter = batch_writer_silver(
    dataframe=silverCleanDF, partition_column="p_ReleaseDate"
)
bronzeToSilverWriter.save(silverPath)

bronzeToSilverGenreWriter = batch_writer_silver_genre_movie_language(
    dataframe=silver_genre
)
bronzeToSilverWriter.save(silverGenrePath)

bronzeToSilverWriter = batch_writer_silver_genre_movie_language(
    dataframe=silver_movie_genre
)
bronzeToSilverWriter.save(silverMovieGenrePath)

bronzeToSilverWriter = batch_writer_silver_genre_movie_language(
    dataframe=silver_originalLanguage
)
bronzeToSilverWriter.save(silverOriginalLanguagePath)

update_bronze_table_status(spark, bronzePath, silverCleanDF, "loaded")
update_bronze_table_status(spark, bronzePath, silverQuarantineDF, "quarantined")

Out[97]: True